# Draft Optimization

This notebook evaluates champion combinations using the local LoL draft prediction model to find the optimal team compositions.
See the showcase for the end result:
https://docs.google.com/document/d/1aHmNZq_Wvn6YChEOKfZa1i1fs_97NExXwBnLYN1WInI/edit?usp=sharing



In [ ]:
import pandas as pd
import requests
import itertools
from tqdm.notebook import tqdm
import os
from dotenv import load_dotenv

from utils.match_prediction.champions import Champion

# Load API key once at the start
load_dotenv()
API_KEY = os.getenv("API_KEY")
HEADERS = {"X-API-Key": API_KEY} if API_KEY else {}

In [ ]:
patch = "15.04"
numerical_elo = 0  # highest numerical elo

## Define Champion Pools

Define the champion pools for each role. Starting with 3 champions per role.


In [ ]:
# Los ratones champion pools
champion_pools = {
    "TOP": [
        "UNKNOWN",
        Champion.JAX,
        Champion.SION,
        Champion.VOLIBEAR,
        Champion.GRAGAS,
        Champion.QUINN,
        Champion.CHOGATH,
        Champion.GAREN,
        Champion.POPPY,
        Champion.VI,
        Champion.AMBESSA,
        Champion.GANGPLANK,
    ],
    "JUNGLE": [
        "UNKNOWN",
        Champion.JARVAN_IV,
        Champion.VIEGO,
        Champion.WUKONG,
        Champion.PANTHEON,
        Champion.VI,
        Champion.IVERN,
        Champion.MAOKAI,
    ],
    "MID": [
        "UNKNOWN",
        Champion.GALIO,
        Champion.ORIANNA,
        Champion.AZIR,
        Champion.SYNDRA,
        Champion.AHRI,
        Champion.VIKTOR,
        Champion.HWEI,
        Champion.MEL,
        Champion.ZILEAN,
        Champion.CHOGATH,
        Champion.VEL_KOZ,
    ],
    "BOT": [
        "UNKNOWN",
        Champion.CORKI,
        Champion.EZREAL,
        Champion.JINX,
        Champion.KALISTA,
        Champion.TRISTANA,
        Champion.ZERI,
        Champion.SIVIR,
        Champion.XAYAH,
        Champion.ZIGGS,
    ],
    "SUPPORT": [
        "UNKNOWN",
        Champion.BRAUM,
        Champion.JANNA,
        Champion.RAKAN,
        Champion.RELL,
        Champion.ALISTAR,
        Champion.RENATA_GLASC,
        Champion.LULU,
        Champion.MILIO,
    ],
}

enemy_champion_pools = {
    "TOP": ["UNKNOWN"],
    "JUNGLE": ["UNKNOWN", Champion.VI, Champion.IVERN, Champion.MAOKAI],
    "MID": ["UNKNOWN"],
    "BOT": ["UNKNOWN", Champion.KALISTA, Champion.EZREAL, Champion.DRAVEN],
    "SUPPORT": [
        "UNKNOWN",
        Champion.BRAUM,
        Champion.BLITZCRANK,
        Champion.THRESH,
        Champion.JANNA,
        Champion.RAKAN,
        Champion.RELL,
        Champion.ALISTAR,
        Champion.RENATA_GLASC,
    ],
}

# Create lookup dictionaries
id_to_name = {champion.id: champion.display_name for champion in Champion}
name_to_id = {champion.display_name: champion.id for champion in Champion}

# Display the champion pools
for role, champions in champion_pools.items():
    champions_display = [
        champion.display_name if isinstance(champion, Champion) else champion
        for champion in champions
    ]
    print(f"ALLY {role}: {', '.join(champions_display)}")

for role, champions in enemy_champion_pools.items():
    champions_display = [
        champion.display_name if isinstance(champion, Champion) else champion
        for champion in champions
    ]
    print(f"ENEMY {role}: {', '.join(champions_display)}")

## Setup Model Prediction Functions

Create functions to get predictions from the model API.


# Batched memory efficient version

In [ ]:
# Don't forget to include the generate_team_comps_stream function and estimate_max_compositions function
def generate_team_comps_stream(
    champion_pools, enemy_champion_pools=None, batch_size=1000
):
    """
    Generate team compositions in batches using a generator to avoid storing all in memory.

    Args:
        champion_pools: Dictionary of champion pools for each role for ally team
        enemy_champion_pools: Dictionary of champion pools for each role for enemy team,
                             or None to use all UNKNOWNs
        batch_size: Number of compositions to yield at once

    Yields:
        Batches of team compositions
    """
    roles = ["TOP", "JUNGLE", "MID", "BOT", "SUPPORT"]

    # Create iterators for all possible combinations
    ally_iter = itertools.product(*[champion_pools[role] for role in roles])

    # If no enemy champion pools provided, use all UNKNOWNs
    if enemy_champion_pools is None:
        enemy_comp = ["UNKNOWN"] * 5

        # Process ally compositions in batches
        batch = []
        for ally_comp in ally_iter:
            # Check for duplicate champions in ally comp
            ally_champion_ids = {
                champion.id for champion in ally_comp if champion != "UNKNOWN"
            }
            if len(ally_champion_ids) == sum(
                1 for champion in ally_comp if champion != "UNKNOWN"
            ):
                batch.append((ally_comp, enemy_comp))

                if len(batch) >= batch_size:
                    yield batch
                    batch = []

        # Yield any remaining compositions
        if batch:
            yield batch
    else:
        # Create iterator for enemy compositions
        enemy_iter = itertools.product(*[enemy_champion_pools[role] for role in roles])
        enemy_comps = []

        # Filter valid enemy compositions first (this is smaller than the cartesian product)
        for comp in enemy_iter:
            champion_ids = {champion.id for champion in comp if champion != "UNKNOWN"}
            if len(champion_ids) == sum(
                1 for champion in comp if champion != "UNKNOWN"
            ):
                enemy_comps.append(
                    (comp, {champion.id for champion in comp if champion != "UNKNOWN"})
                )

        # Process ally compositions in batches
        batch = []
        for ally_comp in ally_iter:
            # Check for duplicate champions in ally comp
            ally_champion_ids = {
                champion.id for champion in ally_comp if champion != "UNKNOWN"
            }
            if len(ally_champion_ids) == sum(
                1 for champion in ally_comp if champion != "UNKNOWN"
            ):

                # Check against each valid enemy comp
                for enemy_comp, enemy_champion_ids in enemy_comps:
                    # Check if there's no overlap between ally and enemy champions
                    if not ally_champion_ids.intersection(enemy_champion_ids):
                        batch.append((ally_comp, enemy_comp))

                        if len(batch) >= batch_size:
                            yield batch
                            batch = []

        # Yield any remaining compositions
        if batch:
            yield batch


def estimate_max_compositions(champion_pools, enemy_champion_pools):
    """
    Calculate the maximum possible number of team compositions (worst case scenario).
    This provides an upper bound for the progress bar.

    Args:
        champion_pools: Dictionary of champion pools for each role for ally team
        enemy_champion_pools: Dictionary of champion pools for each role for enemy team

    Returns:
        Maximum possible number of valid compositions
    """
    roles = ["TOP", "JUNGLE", "MID", "BOT", "SUPPORT"]

    # Count total possible ally compositions
    total_ally_comps = 1
    for role in roles:
        total_ally_comps *= len(champion_pools[role])

    # If no enemy champion pools, just return ally count
    if enemy_champion_pools is None:
        return total_ally_comps

    # Count total possible enemy compositions
    total_enemy_comps = 1
    for role in roles:
        total_enemy_comps *= len(enemy_champion_pools[role])

    # Maximum possible combinations (worst case - no overlaps)
    max_combinations = total_ally_comps * total_enemy_comps

    return max_combinations

In [ ]:
def stream_process_to_sqlite_parallel(
    db_path="lol_team_compositions.db", num_workers=20, commit_frequency=50
):
    """
    Process team compositions in parallel batches and write directly to SQLite
    with optimized database operations.

    Args:
        db_path: Path where the SQLite database will be saved
        num_workers: Number of parallel workers to use for API calls
        commit_frequency: How many batches to process before committing to the database

    Returns:
        Path to the created database
    """
    import sqlite3
    import os
    from tqdm.notebook import tqdm
    import time
    import concurrent.futures
    import queue
    import threading

    # Calculate maximum possible compositions
    max_compositions = estimate_max_compositions(champion_pools, enemy_champion_pools)
    print(f"Maximum possible team compositions: {max_compositions:,}")
    print("Note: Actual number will be lower due to filtering duplicates and overlaps")
    print(f"Using {num_workers} parallel workers for API processing")

    # Calculate batch parameters
    BATCH_SIZE = 1000  # Compositions per batch

    # Estimate number of batches (this is what we'll track in the progress bar)
    estimated_batches = (max_compositions + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"Estimated number of batches: {estimated_batches:,}")

    # Start timing for rate calculation
    start_time = time.time()

    # Remove existing database if it exists
    if os.path.exists(db_path):
        os.remove(db_path)

    # Initialize database with optimized settings
    conn = sqlite3.connect(db_path)

    # Apply SQLite optimizations
    conn.execute(
        "PRAGMA synchronous = OFF"
    )  # Don't wait for writes to be committed to disk
    conn.execute("PRAGMA journal_mode = MEMORY")  # Keep journal in memory
    conn.execute("PRAGMA temp_store = MEMORY")  # Store temp tables in memory
    conn.execute("PRAGMA cache_size = 10000")  # Increase cache size

    # Create champion lookup table
    conn.execute(
        """
    CREATE TABLE champion_lookup (
        id INTEGER PRIMARY KEY,
        name TEXT NOT NULL
    )
    """
    )

    # Insert UNKNOWN champion
    conn.execute("INSERT INTO champion_lookup (id, name) VALUES (0, 'UNKNOWN')")

    # Insert all other champions from our lookup dictionary
    for champ_id, champ_name in id_to_name.items():
        conn.execute(
            "INSERT INTO champion_lookup (id, name) VALUES (?, ?)",
            (int(champ_id), champ_name),
        )

    # Create team_comps table
    conn.execute(
        """
    CREATE TABLE team_comps (
        id INTEGER PRIMARY KEY,
        ally_top_id INTEGER NOT NULL,
        ally_jungle_id INTEGER NOT NULL,
        ally_mid_id INTEGER NOT NULL,
        ally_bot_id INTEGER NOT NULL,
        ally_support_id INTEGER NOT NULL,
        enemy_top_id INTEGER NOT NULL,
        enemy_jungle_id INTEGER NOT NULL,
        enemy_mid_id INTEGER NOT NULL,
        enemy_bot_id INTEGER NOT NULL,
        enemy_support_id INTEGER NOT NULL,
        blue_winrate REAL NOT NULL,
        red_winrate REAL NOT NULL,
        avg_winrate REAL NOT NULL
    )
    """
    )

    # Create role_champions table for filtering
    conn.execute(
        """
    CREATE TABLE role_champions (
        role TEXT NOT NULL,
        champion_id INTEGER NOT NULL,
        PRIMARY KEY (role, champion_id)
    )
    """
    )

    # Insert role champions
    roles = [
        ("ally_top", champion_pools["TOP"]),
        ("ally_jungle", champion_pools["JUNGLE"]),
        ("ally_mid", champion_pools["MID"]),
        ("ally_bot", champion_pools["BOT"]),
        ("ally_support", champion_pools["SUPPORT"]),
        ("enemy_top", enemy_champion_pools["TOP"]),
        ("enemy_jungle", enemy_champion_pools["JUNGLE"]),
        ("enemy_mid", enemy_champion_pools["MID"]),
        ("enemy_bot", enemy_champion_pools["BOT"]),
        ("enemy_support", enemy_champion_pools["SUPPORT"]),
    ]

    for role_name, champions in roles:
        for champion in champions:
            champ_id = (
                0
                if champion == "UNKNOWN"
                else (champion.id if isinstance(champion, Champion) else champion)
            )
            if champ_id != "UNKNOWN":
                champ_id = int(champ_id)
            else:
                champ_id = 0

            conn.execute(
                "INSERT INTO role_champions (role, champion_id) VALUES (?, ?)",
                (role_name, champ_id),
            )

    # Commit initial schema
    conn.commit()
    conn.close()

    # Create queues for producer-consumer pattern
    comp_batch_queue = queue.Queue(
        maxsize=num_workers * 2
    )  # Batches for API processing
    db_batch_queue = queue.Queue(maxsize=num_workers * 2)  # Results for DB insertion

    # Function to process a batch through the API
    def process_batch_api(batch):
        # Create API requests for this batch - separated for blue and red side
        blue_requests = []
        red_requests = []

        for idx, (ally_comp, enemy_comp) in enumerate(batch):
            # Convert champions to IDs, handling UNKNOWN
            ally_ids = [
                "UNKNOWN" if champion == "UNKNOWN" else champion.id
                for champion in ally_comp
            ]
            enemy_ids = [
                "UNKNOWN" if champion == "UNKNOWN" else champion.id
                for champion in enemy_comp
            ]

            # Request for blue side (ally team is on blue side)
            blue_requests.append(
                {
                    "champion_ids": ally_ids + enemy_ids,
                    "numerical_elo": numerical_elo,
                    "patch": patch,
                }
            )

            # Request for red side (ally team is on red side)
            red_requests.append(
                {
                    "champion_ids": enemy_ids + ally_ids,
                    "numerical_elo": numerical_elo,
                    "patch": patch,
                }
            )

        # Make two large API calls instead of many small ones
        blue_results = {}
        red_results = {}

        try:
            # Blue side API call - send all blue requests in one batch
            blue_response = requests.post(
                "http://0.0.0.0:8000/predict-batch",
                json=blue_requests,
                headers=HEADERS,
            )
            blue_response.raise_for_status()

            # Process blue side results
            for i, prediction in enumerate(blue_response.json()):
                blue_results[i] = prediction["win_probability"]

            # Red side API call - send all red requests in one batch
            red_response = requests.post(
                "http://0.0.0.0:8000/predict-batch",
                json=red_requests,
                headers=HEADERS,
            )
            red_response.raise_for_status()

            # Process red side results (invert probability since we want red side win probability)
            for i, prediction in enumerate(red_response.json()):
                red_results[i] = 1 - prediction["win_probability"]

        except Exception as e:
            print(f"Error in batch processing: {e}")

        # Prepare results for database insertion
        rows_to_insert = []
        for idx, (ally_comp, enemy_comp) in enumerate(batch):
            if idx in blue_results and idx in red_results:
                # Convert ally champion IDs
                ally_top_id = 0 if ally_comp[0] == "UNKNOWN" else int(ally_comp[0].id)
                ally_jungle_id = (
                    0 if ally_comp[1] == "UNKNOWN" else int(ally_comp[1].id)
                )
                ally_mid_id = 0 if ally_comp[2] == "UNKNOWN" else int(ally_comp[2].id)
                ally_bot_id = 0 if ally_comp[3] == "UNKNOWN" else int(ally_comp[3].id)
                ally_support_id = (
                    0 if ally_comp[4] == "UNKNOWN" else int(ally_comp[4].id)
                )

                # Convert enemy champion IDs
                enemy_top_id = (
                    0 if enemy_comp[0] == "UNKNOWN" else int(enemy_comp[0].id)
                )
                enemy_jungle_id = (
                    0 if enemy_comp[1] == "UNKNOWN" else int(enemy_comp[1].id)
                )
                enemy_mid_id = (
                    0 if enemy_comp[2] == "UNKNOWN" else int(enemy_comp[2].id)
                )
                enemy_bot_id = (
                    0 if enemy_comp[3] == "UNKNOWN" else int(enemy_comp[3].id)
                )
                enemy_support_id = (
                    0 if enemy_comp[4] == "UNKNOWN" else int(enemy_comp[4].id)
                )

                blue_winrate = blue_results[idx]
                red_winrate = red_results[idx]
                avg_winrate = (blue_winrate + red_winrate) / 2

                rows_to_insert.append(
                    (
                        ally_top_id,
                        ally_jungle_id,
                        ally_mid_id,
                        ally_bot_id,
                        ally_support_id,
                        enemy_top_id,
                        enemy_jungle_id,
                        enemy_mid_id,
                        enemy_bot_id,
                        enemy_support_id,
                        blue_winrate,
                        red_winrate,
                        avg_winrate,
                    )
                )

        # Return stats and rows to insert
        return {
            "compositions_processed": len(batch),
            "compositions_inserted": len(rows_to_insert),
            "api_requests": 2,  # Just two API calls (blue and red)
            "rows": rows_to_insert,
        }

    # API worker function - consumes batches and produces database insertions
    def api_worker():
        while True:
            batch = comp_batch_queue.get()
            if batch is None:  # Termination signal
                comp_batch_queue.task_done()
                break

            result = process_batch_api(batch)
            db_batch_queue.put(result)
            comp_batch_queue.task_done()

    # Database worker function - consumes results and inserts into database
    def db_worker():
        # Create a dedicated database connection for this thread
        conn = sqlite3.connect(db_path)

        # Apply SQLite optimizations
        conn.execute("PRAGMA synchronous = OFF")
        conn.execute("PRAGMA journal_mode = MEMORY")
        conn.execute("PRAGMA temp_store = MEMORY")
        conn.execute("PRAGMA cache_size = 10000")

        # Track how many batches we've processed before committing
        batches_since_commit = 0
        total_rows_inserted = 0

        while True:
            result = db_batch_queue.get()
            if result is None:  # Termination signal
                db_batch_queue.task_done()
                break

            # Insert this batch's rows
            if result["rows"]:
                try:
                    conn.executemany(
                        """
                    INSERT INTO team_comps (
                        ally_top_id, ally_jungle_id, ally_mid_id, ally_bot_id, ally_support_id,
                        enemy_top_id, enemy_jungle_id, enemy_mid_id, enemy_bot_id, enemy_support_id,
                        blue_winrate, red_winrate, avg_winrate
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    """,
                        result["rows"],
                    )

                    total_rows_inserted += len(result["rows"])
                    batches_since_commit += 1

                    # Only commit periodically to improve performance
                    if batches_since_commit >= commit_frequency:
                        conn.commit()
                        batches_since_commit = 0

                except Exception as e:
                    print(f"Error inserting into database: {e}")

            # Record results for progress tracking
            result_queue.put(result)
            db_batch_queue.task_done()

        # Final commit before closing
        if batches_since_commit > 0:
            conn.commit()

        conn.close()
        print(f"Database worker completed, inserted {total_rows_inserted:,} rows total")

    # Create a thread-safe queue for progress reporting
    result_queue = queue.Queue()

    # Create a lock for updating the progress bar
    progress_lock = threading.Lock()

    # Create shared counters for tracking
    total_batches_processed = 0
    total_compositions_processed = 0
    total_compositions_inserted = 0
    total_api_requests = 0

    # Create progress bar for batches
    pbar = tqdm(total=estimated_batches, desc="Processing batches", miniters=1)

    # Function to update progress bar from result queue
    def update_progress():
        nonlocal total_batches_processed, total_compositions_processed
        nonlocal total_compositions_inserted, total_api_requests

        while True:
            try:
                # Get result from queue with timeout
                result = result_queue.get(timeout=1)

                # Update counters
                with progress_lock:
                    total_batches_processed += 1
                    total_compositions_processed += result["compositions_processed"]
                    total_compositions_inserted += result["compositions_inserted"]
                    total_api_requests += result["api_requests"]

                    # Calculate rates
                    elapsed = time.time() - start_time
                    if elapsed > 0:
                        batch_rate = total_batches_processed / elapsed
                        comp_rate = total_compositions_processed / elapsed

                        # Update progress bar description
                        pbar.set_description(
                            f"Processed {total_batches_processed:,} batches "
                            f"({batch_rate:.1f} batches/s, {comp_rate:.1f} comps/s, "
                            f"{total_compositions_inserted:,} inserted)"
                        )

                    # Update progress bar
                    pbar.update(1)

                # Mark task as done
                result_queue.task_done()

            except queue.Empty:
                # Check if processing is complete
                if processing_complete.is_set():
                    break

    # Flag to signal when processing is complete
    processing_complete = threading.Event()

    # Start progress updater thread
    progress_thread = threading.Thread(target=update_progress)
    progress_thread.daemon = True
    progress_thread.start()

    # Start database worker thread
    db_thread = threading.Thread(target=db_worker)
    db_thread.daemon = True
    db_thread.start()

    # Start API worker threads
    api_threads = []
    for _ in range(num_workers):
        thread = threading.Thread(target=api_worker)
        thread.daemon = True
        thread.start()
        api_threads.append(thread)

    # Process batches
    try:
        # Get batch generator
        batch_generator = generate_team_comps_stream(
            champion_pools, enemy_champion_pools, batch_size=BATCH_SIZE
        )

        # Feed batches to API workers
        for batch in batch_generator:
            comp_batch_queue.put(batch)

    except Exception as e:
        print(f"Error generating batches: {e}")
    finally:
        # Signal workers to terminate
        for _ in range(num_workers):
            comp_batch_queue.put(None)

        # Wait for API workers to finish
        for thread in api_threads:
            thread.join()

        # Signal DB worker to terminate
        db_batch_queue.put(None)

        # Wait for DB worker to finish
        db_thread.join()

        # Signal that processing is complete
        processing_complete.set()

        # Wait for progress thread to finish
        progress_thread.join()

        # Close progress bar
        pbar.close()

    # Wait for all results to be processed
    result_queue.join()

    # Create indices for faster querying
    print("Creating indices for faster querying...")
    conn = sqlite3.connect(db_path)
    # Indexes for winrates
    conn.execute("CREATE INDEX idx_avg_winrate ON team_comps(avg_winrate)")
    conn.execute("CREATE INDEX idx_blue_winrate ON team_comps(blue_winrate)")
    conn.execute("CREATE INDEX idx_red_winrate ON team_comps(red_winrate)")

    # Indexes for ally champion IDs
    conn.execute("CREATE INDEX idx_ally_top ON team_comps(ally_top_id)")
    conn.execute("CREATE INDEX idx_ally_jungle ON team_comps(ally_jungle_id)")
    conn.execute("CREATE INDEX idx_ally_mid ON team_comps(ally_mid_id)")
    conn.execute("CREATE INDEX idx_ally_bot ON team_comps(ally_bot_id)")
    conn.execute("CREATE INDEX idx_ally_support ON team_comps(ally_support_id)")

    # Indexes for enemy champion IDs
    conn.execute("CREATE INDEX idx_enemy_top ON team_comps(enemy_top_id)")
    conn.execute("CREATE INDEX idx_enemy_jungle ON team_comps(enemy_jungle_id)")
    conn.execute("CREATE INDEX idx_enemy_mid ON team_comps(enemy_mid_id)")
    conn.execute("CREATE INDEX idx_enemy_bot ON team_comps(enemy_bot_id)")
    conn.execute("CREATE INDEX idx_enemy_support ON team_comps(enemy_support_id)")
    conn.commit()

    # Get the actual count of inserted compositions
    cursor = conn.execute("SELECT COUNT(*) FROM team_comps")
    inserted_count = cursor.fetchone()[0]
    conn.close()

    print(f"SQLite database created at: {db_path}")
    print(
        f"- Created champion_lookup table with {len(id_to_name) + 1} entries (including UNKNOWN)"
    )
    print(
        f"- Processed {total_batches_processed} batches ({total_compositions_processed:,} compositions)"
    )
    print(f"- Made {total_api_requests:,} API requests")
    print(f"- Inserted {inserted_count:,} team compositions into database")
    print(f"- All champion IDs are stored as integers (with UNKNOWN as 0)")

    return db_path


def estimate_max_compositions(champion_pools, enemy_champion_pools):
    """
    Calculate the maximum possible number of API requests (worst case scenario).
    This provides an upper bound for the progress bar.

    Args:
        champion_pools: Dictionary of champion pools for each role for ally team
        enemy_champion_pools: Dictionary of champion pools for each role for enemy team

    Returns:
        Maximum possible number of API requests (2 per composition)
    """
    roles = ["TOP", "JUNGLE", "MID", "BOT", "SUPPORT"]

    # Count total possible ally compositions
    total_ally_comps = 1
    for role in roles:
        total_ally_comps *= len(champion_pools[role])

    # If no enemy champion pools, just return ally count
    if enemy_champion_pools is None:
        # Multiply by 2 because we make 2 API requests per composition (blue and red side)
        return total_ally_comps * 2

    # Count total possible enemy compositions
    total_enemy_comps = 1
    for role in roles:
        total_enemy_comps *= len(enemy_champion_pools[role])

    # Maximum possible combinations (worst case - no overlaps)
    # Multiply by 2 because we make 2 API requests per composition (blue and red side)
    max_api_requests = total_ally_comps * total_enemy_comps * 2

    return max_api_requests


db_path = stream_process_to_sqlite_parallel(num_workers=20)

In [ ]:
# Create indices for faster querying
import sqlite3

print("Creating indices for faster querying...")
conn = sqlite3.connect(db_path)

# New indexes for other winrates
conn.execute("CREATE INDEX idx_blue_winrate ON team_comps(blue_winrate)")
conn.execute("CREATE INDEX idx_red_winrate ON team_comps(red_winrate)")

# Indexes for ally champion IDs
conn.execute("CREATE INDEX idx_ally_top ON team_comps(ally_top_id)")
conn.execute("CREATE INDEX idx_ally_jungle ON team_comps(ally_jungle_id)")
conn.execute("CREATE INDEX idx_ally_mid ON team_comps(ally_mid_id)")
conn.execute("CREATE INDEX idx_ally_bot ON team_comps(ally_bot_id)")
conn.execute("CREATE INDEX idx_ally_support ON team_comps(ally_support_id)")

# Indexes for enemy champion IDs
conn.execute("CREATE INDEX idx_enemy_top ON team_comps(enemy_top_id)")
conn.execute("CREATE INDEX idx_enemy_jungle ON team_comps(enemy_jungle_id)")
conn.execute("CREATE INDEX idx_enemy_mid ON team_comps(enemy_mid_id)")
conn.execute("CREATE INDEX idx_enemy_bot ON team_comps(enemy_bot_id)")
conn.execute("CREATE INDEX idx_enemy_support ON team_comps(enemy_support_id)")

conn.commit()